# ライブラリ

In [2]:
import numpy as np
import csv
import torch     
import pandas as pd
import math
import pprint
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from tqdm import trange
from tqdm.contrib import tzip,tqdm,tenumerate
#from tqdm.notebook import tqdm
from IPython.display import display


from collections import OrderedDict

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
output_path="density_3d_arrray/density_2410_quarter_highresidual.pt"
output_csv_path="output2410_b200.csv"

In [4]:
config="config/config0703_nozzle"#.csv

#処理タスクリストを取得
df_config=pd.read_csv("{}.csv".format(str(config)))
df_config=df_config.set_index("experiment")
df_config=df_config[df_config["process_flag"]==1]#処理タスクリストから処理フラグ有効のみを残す

# ラプラシアンを計算

In [5]:
path="reconstructed/reconstructed2410_res9.npy"
array_nabura=np.load(path)
print(array_nabura.shape)
print(array_nabura)

(557, 371, 371)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 

In [6]:
def compute_laplacian_chunk(array_chunk):
    grad_yy = np.gradient(array_chunk, axis=1)
    grad_zz = np.gradient(array_chunk, axis=2)
    laplacian_chunk = grad_yy + grad_zz
    return laplacian_chunk

def compute_laplacian_in_chunks(array, chunk_size):
    # 配列の形状を取得
    shape = array.shape
    
    # 結果を保存する配列を作成
    laplacian = np.zeros_like(array)
    
    # チャンクごとに処理
    for i in trange(0, shape[0], chunk_size):
        for j in range(0, shape[1], chunk_size):
            for k in range(0, shape[2], chunk_size):
                # チャンクを抽出
                chunk = array[i:i+chunk_size, j:j+chunk_size, k:k+chunk_size]
                
                # チャンクでラプラシアンを計算
                laplacian_chunk = compute_laplacian_chunk(chunk)
                
                # 計算結果を元の配列の対応する位置に保存
                laplacian[i:i+chunk_size, j:j+chunk_size, k:k+chunk_size] = laplacian_chunk
    
    return laplacian

# チャンクサイズを指定して計算
chunk_size = 100
array_laplacian = compute_laplacian_in_chunks(array_nabura, chunk_size)


100%|██████████| 6/6 [00:00<00:00, 18.45it/s]


# データローダーをつくる

In [7]:
batch_size=1#ミニバッチのサイズを指定
array_laplacian_dataloarder=torch.utils.data.DataLoader(torch.tensor(array_laplacian).to(device),batch_size=batch_size,shuffle=False)#シャッフルは切っておく

# SOR法

In [8]:
Lx=array_laplacian.shape[0]
Ly=array_laplacian.shape[1]
Lz=array_laplacian.shape[2]
x  = Lx
y  = Ly
z  = Lz

In [1]:
e = 1e-9
tolerance = 1e-24
max_stable_iters = 1000000

# メイン
delta = 1.0
n_iter = 0
stable_count = 0
prev_delta = float('inf')
u_list = []
omega_SOR=1.0

# tqdmのインスタンスを一番上に表示
#progress_bar = tqdm(total=len(array_laplacian_dataloarder))
#display(progress_bar.container)


# 全バッチの結果を保存するリスト
all_batch_results = []


for batch_idx, batch in tenumerate(array_laplacian_dataloarder):
    slice_laplacian = batch.to(device)  # バッチデータをデバイスに移動
    batch_size, Ly, Lz = slice_laplacian.size()
    u = torch.zeros([batch_size, Ly, Lz], device=device)  # uを同じデバイスで作成
    delta = 1.0
    n_iter = 0
    stable_count = 0  # 安定回数のカウンタをリセット

    # グラフ用のリストを初期化
    loss_list = []
    u_max_list = []
    u_min_list = []
    iter_list=[]
    max_iterations =0

    while delta > e and stable_count < max_stable_iters:
        u_in = u.clone()
        u[:, 1:-1, 1:-1] =  u[:, 1:-1, 1:-1]+ omega_SOR*((u_in[:, 2:, 1:-1] + u_in[:, :-2, 1:-1] +
                            u_in[:, 1:-1, 2:] + u_in[:, 1:-1, :-2] + slice_laplacian[:, 1:-1, 1:-1]) / 4- u[:, 1:-1, 1:-1])
        
        u[:, 0, :] = 0
        u[:, Ly-1, :] = 0
        u[:, :, 0] = 0
        u[:, :, Lz-1] = 0
        delta = torch.max(torch.abs(u - u_in))
        
        # 残差の変化をチェック
        if abs(delta - prev_delta) < tolerance:
            stable_count += 1
        else:
            stable_count = 0

        prev_delta = delta

        print("\r",f'Iteration: {n_iter}, Loss: {delta}, Stable Count: {stable_count}',end="")
        # イテレーションごとのmaxとminを記録
        u_max = torch.max(u).item()
        u_min = torch.min(u).item()
        loss_list.append(delta.cpu().float().item())
        u_max_list.append(u_max)
        u_min_list.append(u_min)
        iter_list.append(n_iter)

        n_iter += 1

    # バッチごとの結果を保存
    all_batch_results.append((batch_idx + 1, loss_list, u_max_list, u_min_list,iter_list))

    
    #progress_bar.update(1)
    u_list.append(u)

u_tensor = torch.cat(u_list, dim=0)  # 最終的な出力をリストからテンソルに変換


  0%|          | 0/557 [00:00<?, ?it/s]

 Iteration: 1921, Loss: 4.487446858547628e-09, Stable Count: 009

In [9]:
import csv
# CSVファイルに保存
with open(output_csv_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(all_batch_results)

In [10]:
for batch_idx, iter_list, u_max_list, u_min_list in all_batch_results:
    fig, ax = plt.subplots(figsize=(10, 5))
    

    iter_list_cpu_filtered = [x for x in iter_list if x > 0]
    u_max_list_filtered = [u_max_list[i] for i in range(len(iter_list)) if iter_list[i] > 0]
    u_min_list_filtered = [u_min_list[i] for i in range(len(iter_list)) if iter_list[i] > 0]


    # 点をプロット
    ax.scatter(iter_list_cpu_filtered, u_max_list_filtered, label='Max Value', color='r')
    ax.scatter(iter_list_cpu_filtered, u_min_list_filtered, label='Min Value', color='b')
    
    ax.set_xlim(max(iter_list_cpu_filtered), min(iter_list_cpu_filtered))  # X軸の範囲を逆に設定して反転
    ax.set_xscale('log')  # X軸を対数スケールに設定
    ax.set_xlabel('residual (Log Scale)')
    ax.set_ylabel('Max/Min refractive index')
    ax.set_title(f'Batch {batch_idx} - Iteration vs Max/Min refractive index')
    ax.legend()
    ax.grid(True)

    plt.tight_layout()
    plt.show()

# 密度の計算

In [12]:
print(u_tensor.shape)

In [13]:
exp=df_config.index[0]
temperature=df_config["temperature(℃)"][exp]
pressure=df_config["pressure(hPa)"][exp]
humidity=df_config["humidity(%)"][exp] #湿度
density_inf=df_config["density_inf"][exp]

n_inf=1.0003
G=(n_inf-1)/density_inf#Gladstone-Dale Relation
print(G)

In [14]:
d_density=-u_tensor/G
density=d_density+density_inf
print(density)
torch.save(density, output_path)

In [15]:
middle_index=int((density.shape[1])/2)
density_s=density[:,middle_index,:]
density_s_wide=np.rot90(np.array(density_s.cpu()),1)
print(density_s_wide.shape)

def plot_showandsave(array,path,name):
    # ヒートマップを作成
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 6)) # 図の設定
        #c = ax.pcolor(array, cmap='binary_r') # ヒートマップ
        c = ax.pcolor(array, cmap='binary_r',norm=Normalize(vmin=0, vmax=3)) # ヒートマップ


        ax.set_title(name) # タイトル
        ax.set_aspect('equal', adjustable='box') # アスペクト比
        fig.colorbar(c, ax=ax) # カラーバー
        plt.savefig("{}\{}".format(path,name))#画像の保存
        print("ヒートマップの保存")
        plt.show() # 図を表示
        print("ヒートマップの表示")

plot_showandsave(density_s_wide,"density","densitymap")

In [16]:
density_s

In [17]:
plt.hist(density_s.cpu(),density=True)
plt.xlim(0,3)
plt.show()